# Data Preprocessing for PetFinder6000

In [1]:
import pandas as pd
import numpy as np
import re
import os
import glob
import shutil

import boto3
import sagemaker

## Users

## Load Data from S3

In [2]:
%env AWS_PROFILE=aeroxye-sagemaker

env: AWS_PROFILE=aeroxye-sagemaker


In [3]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1687399683",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1687399683"
}


In [4]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)

In [5]:
region = boto3.Session().region_name

data_bucket = "dynamodbpetfinder"
object_prefix = "user_"
local_path = "../data/user/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)

pattern = r'[0-9]+'
s3 = boto3.client("s3")

result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
subfolders.sort(reverse=True)

object_path = object_prefix + subfolders[0] + "/"
files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

pattern = rf'{object_path}(.+)'
for content in files.get("Contents"):
    file_path = content.get("Key")
    filename = re.findall(pattern, file_path)[0]
    print(filename)

    with open(local_path+filename, 'wb') as file:
        s3.download_fileobj(
            Bucket=data_bucket,
            Key=file_path,
            Fileobj=file
        )

run-1686726799591-part-r-00000
run-1686726799591-part-r-00001
run-1686726799591-part-r-00002
run-1686726799591-part-r-00003


In [6]:
file_list = glob.glob(local_path + "*")

print(file_list)

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_json(file, lines=True) # read data frame from json file
    dfs.append(data) # append the data frame to the list
    
users = pd.concat(dfs, ignore_index=True)

['../data/user\\run-1686726799591-part-r-00000', '../data/user\\run-1686726799591-part-r-00001', '../data/user\\run-1686726799591-part-r-00002', '../data/user\\run-1686726799591-part-r-00003']


In [7]:
users.head()

,id,__typename,A_othercats,A_sweetspicy,_lastChangedAt,A_agegroup,A_gender,A_otherdogs,A_employment,createdAt,...,A_firstcat,A_kids,_version,A_attention,updatedAt,A_primarycolor,username,A_homeownership,A_allergies,A_energy
0,15c83d3c-196c-4f57-bee2-361ac6fcc21e,User,-1,Anything is nice,1684463702517,[{'S': 'Kitten'}],No preference,1,Working Full Time,2023-05-19T02:35:02.489Z,...,1,1,1,Independent,2023-05-19T02:35:02.489Z,[{'S': 'No preference'}],Maxmaxmax,Staying with Parents,1,Chill
1,8fe87a5a-a6cf-49db-8b7a-fca9772a717c,User,-1,Some spice,1684297951088,"[{'S': 'Kitten'}, {'S': 'Juvenile'}]",No preference,-1,Student,2023-05-17T04:32:31.057Z,...,1,-1,1,Independent,2023-05-17T04:32:31.057Z,[{'S': 'No preference'}],biondibiondi,Staying with Parents,-1,Chill
2,1528304d-e070-469a-b476-31f3b1022681,User,-1,Anything is nice,1684389123230,[{'S': 'No preference'}],No preference,-1,Working Full Time,2023-05-18T05:52:03.205Z,...,1,-1,1,No preference,2023-05-18T05:52:03.205Z,[{'S': 'No preference'}],Xiaozha1,Self-Owned,-1,No preference
3,bb6d45b5-f368-4877-9a5f-b3cbb8761bd7,User,-1,Anything is nice,1685688338287,[{'S': 'Kitten'}],No preference,-1,Working Full Time,2023-06-02T06:45:38.257Z,...,1,-1,1,Independent,2023-06-02T06:45:38.257Z,[{'S': 'No preference'}],renjie25,Staying with Parents,-1,Chill
4,9e936f8b-4d16-4d19-b864-6262b350f44f,User,-1,Anything is nice,1684553605346,"[{'S': 'Kitten'}, {'S': 'Juvenile'}, {'S': 'Ad...",No preference,-1,Working Full Time,2023-05-20T03:33:25.319Z,...,1,-1,1,Independent,2023-05-20T03:33:25.319Z,"[{'S': 'Black'}, {'S': 'Calico/Tortie'}, {'S':...",Blubelli,Staying with Parents,-1,Chill


## Process Data

In [8]:
# rename headers
cl_users = users.rename(columns={'A_gender': 'gender',
                              'A_primarycolor': 'primary_color',
                              'A_agegroup': 'age_group',
                              'A_energy': 'energy_level',
                              'A_attention': 'attention_need',
                              'A_sweetspicy': 'personality',
                              'A_firstcat': 'is_first_cat',
                              'A_othercats': 'has_other_cats',
                              'A_otherdogs': 'good_with_other_dogs',
                              'A_kids': 'good_with_kids',
                              'A_employment': 'employment',
                              'A_homeownership': 'home_ownership',
                              'A_allergies': 'has_allergies',
                              'A_adoptionfee': 'agree_to_fee',
                              'createdAt': 'created_at',
                              'updatedAt': 'updated_at',
                              })

In [9]:
# clean multi-select columns with No Preference options (age, color)

def clean_multi_select(row):
    arr = [o.get("S") for o in row]
    if (len(arr) > 1) and ('No preference' in arr):
        arr.remove('No preference')
    return [s.lower() for s in arr]

cl_users['age_group'] = cl_users['age_group'].map(lambda choice: clean_multi_select(choice))
cl_users['primary_color'] = cl_users['primary_color'].map(lambda choice: clean_multi_select(choice))

In [10]:
# split columns with list (age, color)
age_groups = cl_users['age_group'].explode().unique().tolist()
split_age_groups = cl_users['age_group'].map(lambda row: ','.join([str(age in row) for age in age_groups]))
new_age_columns = split_age_groups.str.split(',', expand=True)
new_age_columns = new_age_columns.applymap(lambda val: (val == 'True'))
# new_age_columns = new_age_columns.astype('bool')
new_age_columns = new_age_columns.astype('int')

pattern = re.compile(r'\s|/')
# new_age_columns.columns = [f'age_{pattern.sub("_", age).lower()}' for age in age_groups]
cl_users[[f'age_{pattern.sub("_", age).lower()}' for age in age_groups]] = new_age_columns
cl_users = cl_users.drop('age_group', axis=1)

color_groups = cl_users['primary_color'].explode().unique().tolist()
split_color_groups = cl_users['primary_color'].map(lambda row: ','.join([str(color in row) for color in color_groups]))
new_color_columns = split_color_groups.str.split(',', expand=True)
new_color_columns = new_color_columns.applymap(lambda val: (val == 'True'))
# new_color_columns = new_color_columns.astype('bool')
new_color_columns = new_color_columns.astype('int')

cl_users[[f'primary_color_{pattern.sub("_", color).lower()}' for color in color_groups]] = new_color_columns
cl_users = cl_users.drop('primary_color', axis=1)

In [11]:
# convert string fields to lower case (gender, energy_level, attention_need, personality, employment, home_ownership)
cl_users['gender'] = cl_users['gender'].map(lambda val: val.lower())
cl_users['energy_level'] = cl_users['energy_level'].map(lambda val: val.lower())
cl_users['attention_need'] = cl_users['attention_need'].map(lambda val: val.lower())
cl_users['personality'] = cl_users['personality'].map(lambda val: val.lower())
cl_users['employment'] = cl_users['employment'].map(lambda val: val.lower())
cl_users['home_ownership'] = cl_users['home_ownership'].map(lambda val: val.lower())

In [12]:
# # convert int booleans to actual booleans (is_first_cat, has_other_cats, good_with_other_dogs, good_with_kids, has_allergies, agree_to_fee)
cl_users['is_first_cat'] = cl_users['is_first_cat'].map(lambda val: (val == 1))
cl_users['has_other_cats'] = cl_users['has_other_cats'].map(lambda val: (val == 1))
cl_users['good_with_other_dogs'] = cl_users['good_with_other_dogs'].map(lambda val: (val == 1))
cl_users['good_with_kids'] = cl_users['good_with_kids'].map(lambda val: (val == 1))
cl_users['has_allergies'] = cl_users['has_allergies'].map(lambda val: (val == 1))
cl_users['agree_to_fee'] = cl_users['agree_to_fee'].map(lambda val: (val == 1))

In [13]:
# convert types
# cl_users = cl_users.astype({'gender': 'category',
#                           'energy_level': 'category',
#                           'attention_need': 'category',
#                           'personality': 'category',
#                           'is_first_cat': 'bool',
#                           'has_other_cats': 'bool',
#                           'good_with_other_dogs': 'bool',
#                           'good_with_kids': 'bool',
#                           'employment': 'category',
#                           'home_ownership': 'category',
#                           'has_allergies': 'bool',
#                           'agree_to_fee': 'bool',
#                           })
cl_users['created_at'] = pd.to_datetime(cl_users['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
cl_users['updated_at'] = pd.to_datetime(cl_users['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

cl_users = cl_users.astype({
                          'is_first_cat': 'int',
                          'has_other_cats': 'int',
                          'good_with_other_dogs': 'int',
                          'good_with_kids': 'int',
                          'has_allergies': 'int',
                          'agree_to_fee': 'int',
                          'created_at': 'object',
                          'updated_at': 'object',
                          })


In [13]:
# cl_users.columns

In [14]:
# user_pref = pd.DataFrame()
# user_pref['id'] = cl_users['id']

In [15]:
# user_pref_cols = ['gender', 'primary_color', 'age_group',
#        'energy_level', 'attention_need', 'personality', 'is_first_cat',
#        'has_other_cats', 'good_with_other_dogs', 'good_with_kids',
#        'employment', 'home_ownership', 'has_allergies', 'agree_to_fee']
#
# user_pref['preference'] = cl_users.apply(lambda row: '\n'.join([f'{pref}: {row[pref]}' for pref in user_pref_cols]), axis=1)

In [16]:
# user_pref.head(5)

In [14]:
# drop glue columns
cl_users = cl_users.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

In [15]:
cl_users = cl_users.set_index('id')

In [16]:
cl_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 15c83d3c-196c-4f57-bee2-361ac6fcc21e to 029f1e2a-d68b-4801-869f-f422e925ae3d
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   has_other_cats               107 non-null    int32 
 1   personality                  107 non-null    object
 2   gender                       107 non-null    object
 3   good_with_other_dogs         107 non-null    int32 
 4   employment                   107 non-null    object
 5   created_at                   107 non-null    object
 6   agree_to_fee                 107 non-null    int32 
 7   is_first_cat                 107 non-null    int32 
 8   good_with_kids               107 non-null    int32 
 9   attention_need               107 non-null    object
 10  updated_at                   107 non-null    object
 11  username                     107 non-null    object
 12  home_ownership               

In [17]:
cl_users.head()

,has_other_cats,personality,gender,good_with_other_dogs,employment,created_at,agree_to_fee,is_first_cat,good_with_kids,attention_need,...,age_no_preference,age_adult,age_senior,primary_color_no_preference,primary_color_black,primary_color_calico_tortie,primary_color_tabby,primary_color_others,primary_color_ginger,primary_color_white
id,,,,,,,,,,,,,,,,,,,,,
15c83d3c-196c-4f57-bee2-361ac6fcc21e,0,anything is nice,no preference,1,working full time,2023-05-19T02:35:02Z,0,1,1,independent,...,0,0,0,1,0,0,0,0,0,0
8fe87a5a-a6cf-49db-8b7a-fca9772a717c,0,some spice,no preference,0,student,2023-05-17T04:32:31Z,1,1,0,independent,...,0,0,0,1,0,0,0,0,0,0
1528304d-e070-469a-b476-31f3b1022681,0,anything is nice,no preference,0,working full time,2023-05-18T05:52:03Z,1,1,0,no preference,...,1,0,0,1,0,0,0,0,0,0
bb6d45b5-f368-4877-9a5f-b3cbb8761bd7,0,anything is nice,no preference,0,working full time,2023-06-02T06:45:38Z,1,1,0,independent,...,0,0,0,1,0,0,0,0,0,0
9e936f8b-4d16-4d19-b864-6262b350f44f,0,anything is nice,no preference,0,working full time,2023-05-20T03:33:25Z,1,1,0,independent,...,0,1,0,0,1,1,1,0,0,0


## Export Processed Data as csv to Local Folder

In [18]:
processed_folder = "../data/processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cl_users.to_csv(f'{processed_folder}users.csv')